In [1]:
import pymortar
import glob
URL = "http://mortar_mortar-server_1:5001"
# connect client to Mortar frontend server
c = pymortar.Client(URL)

In [2]:
# query AHU, supply air temp sensor, and downstream zone
query1 = """SELECT ?ahu ?sat ?zone WHERE {
    ?ahu a brick:AHU .
    ?ahu brick:hasPart?/brick:hasPoint ?sat .
    ?sat a brick:Supply_Air_Temperature_Sensor .
    ?ahu brick:feeds+ ?zone .
    ?zone a brick:HVAC_Zone
}"""

# damper position commands in VAVs
query2 = """SELECT ?vav ?cmd WHERE {
    ?vav   a  brick:VAV .
    ?vav   brick:hasPart?/brick:hasPoint ?cmd .
    ?cmd a brick:Damper_Position_Command
}"""

# all air temperature sensors
query3 = """SELECT ?ats WHERE {
    ?ats  a  brick:Air_Temperature_Sensor .
}"""

## Querying (Brick) Metadata

In [3]:
# execute query against all sites
# c.sparql(query)

# or against a list of sites
c.sparql(query1, sites=['bldg1'])

,ahu,sat,zone,site
0,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM091_LAB,bldg1
1,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM190_LAB,bldg1
2,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM280F_LAB,bldg1
3,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM180_LAB,bldg1
4,http://buildsys.org/ontologies/BLDG1#AHU07A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM285_LAB,bldg1
...,...,...,...,...
389,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM034_LAB,bldg1
390,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM041_LAB,bldg1
391,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM246_LAB,bldg1
392,http://buildsys.org/ontologies/BLDG1#AHU04A,http://buildsys.org/ontologies/BLDG1#BLDG1.AHU...,http://buildsys.org/ontologies/BLDG1#RM148_LAB,bldg1


In [4]:
# get a summary of which sites return results for a set of queries
c.qualify([query1, query2, query3])

       Query_0  Query_1  Query_2
bldg1      394        0      829
bldg2        0        0        2

In [5]:
# name the queries (optional)
c.qualify({"damper_cmd": query2, "ahu sat": query1, "temp sensors": query3})

       damper_cmd  ahu sat  temp sensors
bldg1           0      394           829
bldg2           0        0             2

## Querying Timeseries Data

In [6]:
# get SPARQL query results, stream metadata data tables as separate objects
res = c.data_sparql(query3)

In [7]:
# sparql query results
res.metadata

,ats
0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
1,http://example.com#mysensortest
2,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
3,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
4,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
...,...
826,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
827,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
828,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
829,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...


In [8]:
# stream metadata
res.streams

,brick_class,brick_uri,units,name,stream_id
0,https://brickschema.org/schema/Brick#Zone_Air_...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU04B.RM149C_LAB.Zone Air Temp,251
1,https://brickschema.org/schema/Brick#Zone_Air_...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU01A.RM204A_LAB.Zone Air Temp,1274
2,https://brickschema.org/schema/Brick#Zone_Air_...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU01A.RM104A_LAB.Zone Air Temp,139
3,https://brickschema.org/schema/Brick#Supply_Ai...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU04B.RM049_LAB.Zone Supply Air Temp,1245
4,https://brickschema.org/schema/Brick#Zone_Air_...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU02B.RM224A_LAB.Zone Air Temp,1260
...,...,...,...,...,...
822,https://brickschema.org/schema/Brick#Supply_Ai...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU07A.RM280H_LAB.Zone Supply Air Temp,904
823,https://brickschema.org/schema/Brick#Supply_Ai...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU02A.RM115_A_LAB.Zone Supply Air ...,1185
824,https://brickschema.org/schema/Brick#Zone_Air_...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU05A.RM366_LAB.Zone Air Temp,1046
825,https://brickschema.org/schema/Brick#Supply_Ai...,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...,degF,BLDG1.ZONE.AHU02A.RM214_LAB.Zone Supply Air Temp,560


In [9]:
# actual timeseries data
res.data

,time,value,id
0,2015-01-01 00:00:00+00:00,69.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
1,2015-01-01 00:15:00+00:00,64.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
2,2015-01-01 00:30:00+00:00,69.5,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
3,2015-01-01 00:45:00+00:00,61.5,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
4,2015-01-01 01:00:00+00:00,61.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
...,...,...,...
158399,2015-01-02 23:45:00+00:00,74.0,http://buildsys.org/ontologies/BLDG1#BLDG1.ZON...
158400,2015-01-01 00:00:00+00:00,72.0,http://example.com#mysensortest
158401,2015-01-01 00:10:00+00:00,72.3,http://example.com#mysensortest
158402,2015-01-01 00:20:00+00:00,72.4,http://example.com#myothersensor


In [10]:
# get dataset with URIs
res = c.data_uris(["http://example.com#mysensortest", "http://example.com#myothersensor"])
res.streams

,brick_class,brick_uri,units,name,stream_id
0,https://brickschema.org/schema/Brick#Air_Tempe...,http://example.com#myothersensor,degF,myothersensor,1286
1,https://brickschema.org/schema/Brick#Air_Tempe...,http://example.com#mysensortest,degF,mysensortest,1285


In [11]:
res.data

,time,value,id
0,2015-01-01 00:00:00+00:00,72.0,http://example.com#mysensortest
1,2015-01-01 00:10:00+00:00,72.3,http://example.com#mysensortest
2,2015-01-01 00:20:00+00:00,72.4,http://example.com#myothersensor
3,2015-01-01 00:30:00+00:00,73.0,http://example.com#myothersensor
